In [1]:
# Hacking Google Finance in Real-Time for Algorithmic Traders
# 
# (c) 2014 QuantAtRisk.com, by Pawel Lachowicz
# Python-3 version --- urllib.request used instead of urllib2
# Link to related article:
# http://www.quantatrisk.com/2014/01/14/hacking-google-finance-in-real-time-for-algorithmic-traders/

import urllib.request, time, os, re, csv

In [2]:
def fetchGF(googleticker):
    url = "http://www.google.com/finance?&q="
    req = urllib.request.Request(url + googleticker)
    resp = urllib.request.urlopen(req)
    respData = resp.read()
    k=re.search(b'id="ref_(.*?)">(.*?)<', respData)
    if k:
        tmp=k.group(2)
        q=tmp.decode().replace(',','')
    else:
        q="Nothing found for: "+googleticker
    return q

In [5]:
# display time corresponding to your location
print(time.ctime())
print()
 
# Set local time zone to NYC
os.environ['TZ']='America/New_York'
time.tzset()
t=time.localtime() # string
print(time.ctime())
print()

Thu May 14 08:58:25 2015

Thu May 14 08:58:25 2015



In [6]:
def combine(ticker):
    quote=fetchGF(ticker) # use the core-engine function
    t=time.localtime()    # grasp the moment of time
    output=[t.tm_year,t.tm_mon,t.tm_mday,t.tm_hour,  # build a list
            t.tm_min,t.tm_sec,ticker,quote]
    return output

In [7]:
ticker="NASDAQ:AAPL"

In [8]:
# define file name of the output record
fname="aapl.dat"
# remove a file, if exist
os.path.exists(fname) and os.remove(fname)

In [ ]:
# To test, set the hour and minutes according to the above printed
# time in New York so you can capture 2 or 3 values before the loop
# stops

freq=600 # fetch data every 600 sec (10 min)

tickers=["NASDAQ:AAPL","NASDAQ:GOOG","NASDAQ:BIDU","NYSE:IBM", \
         "NASDAQ:INTC","NASDAQ:MSFT","NYSEARCA:SPY"]
 
with open(fname,'a') as f:
    writer = csv.writer(f,dialect="excel") #,delimiter=" ")
    while(t.tm_hour <= 9):
        if(t.tm_hour == 9):
            while(t.tm_min < 31):
                data = combine(ticker)
                print(data)
                writer.writerow(data) # save data in the file
                time.sleep(freq)
            else:break
        else:
            for ticker in tickers:
                data=combine(ticker)
                print(data)
                writer.writerow(data) # save data in the file
                time.sleep(freq)
f.close()
print(str(t.tm_hour) + ":" + str(t.tm_min))

[2015, 5, 14, 8, 58, 30, 'NASDAQ:AAPL', '4378.31']
[2015, 5, 14, 9, 8, 30, 'NASDAQ:GOOG', '4378.31']
[2015, 5, 14, 9, 18, 31, 'NASDAQ:BIDU', '4378.31']
[2015, 5, 14, 9, 28, 31, 'NYSE:IBM', '4378.31']
[2015, 5, 14, 9, 38, 32, 'NASDAQ:INTC', '4378.31']